<a href="https://colab.research.google.com/github/fatday/My-Grad-Math-Works/blob/main/CME/A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# Q1

For homogeneous solution:

$$
-u''+u=0 \ \ \ \ \implies \ \ \ \ \lambda_1=1 \ \ \ \lambda_2=-1 \ \ \ \ \implies \ \ \ u_h(x)=C_1e^x+C_2e^{-x}
$$

Now we solve for particular solution, let $u_p(x)=Ax+B$, then

$$
-u''+u=2x \ \ \ \ \implies \ \ \ \ Ax+B=2x \ \ \ \ \implies \ \ \ \ u_p(x)=2x
$$

Then we have the solution

$$
u(x)=u_h(x)+u_p(x)=C_1e^x+C_2e^{-x}+2x
$$

Since $u(0)=u(1)=0$, then solving $C_1,C_2$ we get

$$
u(x)=\frac{2e}{1-e^2}e^x-\frac{2e}{1-e^2}e^{-x}+2x
$$

In [3]:
def exact(h, X_range):
  x = np.array([i * h for i in range(1, int((X_range[1] - X_range[0])/h))])
  coef = (2 * np.e) / (1 - np.e**2)
  return coef * np.e**x - coef * np.e**(-x) + 2*x

In [4]:
def f(x):
  return 2*x

In [5]:
def solve_solution(h,X_range):
  x_vec = np.array([i * h for i in range(1, int((X_range[1] - X_range[0])/h))])
  n = len(x_vec)
  a,b = (2/h) + (2*h/3), (h/6 - 1/h)
  A = ((np.diag(a * np.ones(n), 0)  + np.diag(b * np.ones(n-1), 1) + np.diag(b * np.ones(n-1), -1)))
  fx =  f(x_vec) * h  # (f, Chi) = int f_i * Chi_i=int f_i on [0,1] = f_i
  u = np.linalg.solve(A, fx)
  #print(A)
  return u

In [6]:
X_range = [0, 1]
h1, h2 = 1/10, 1/20
U1 = solve_solution(h1, X_range)
U2 = solve_solution(h2, X_range)

In [7]:
u1 = exact(h1, X_range)
u2 = exact(h2, X_range)

## Max Error at mesh-points for h= 1/10, 1/20

For $h=1/10$: max error is $8.8514\cdot 10^{-5}$

In [8]:
max(abs(U1-u1))

8.85143578538139e-05

For $h=1/20$: max error is  $2.2108\cdot 10^{-5}$

In [9]:
max(abs(U2-u2))

2.2107691947451102e-05

# Q2

In [116]:
def f(x1, x2):
  return np.sin(np.pi * x1) * (np.sin(np.pi * x2) + np.sin(2 * np.pi * x2))


def u(x1, x2):
  coef1, coef2 = 1 / (2 * (np.pi)**2), 1 / (5 * (np.pi)**2)
  base1, base2 = np.sin(np.pi * x1) * np.sin(np.pi * x2), np.sin(np.pi * x1) * np.sin(2 * np.pi * x2)
  return coef1 * base1 + coef2 * base2

In [182]:
def load_vec_lattices(i, j, h, n):
  p0 = np.array([i * h, j * h])

  # ver
  p1 = np.array([(i+1) * h, j * h])
  p2 = np.array([(i - 1) * h, j * h])

  # horz
  p3 = np.array([i * h, (j+1) * h])
  p4 = np.array([i * h, (j-1) * h])

  # diag
  p5 = np.array([(i-1) * h, (j+1) * h])
  p6 = np.array([(i+1) * h, (j-1) * h])

  # tri_mid
  t1 = np.mean([p0, p2, p4], axis=0)
  t2 = np.mean([p0, p2, p5], axis=0)
  t3 = np.mean([p0, p3, p5], axis=0)
  t4 = np.mean([p0, p4, p6], axis=0)
  t5 = np.mean([p0, p1, p6], axis=0)
  t6 = np.mean([p0, p1, p3], axis=0)

  tris =  np.array([t1, t2, t3, t4, t5, t6])
  f_val = sum(f(tris[:, 0], tris[:, 1]))
  return f_val / 3

In [202]:

def Finite_Difference_Approx(h, space):
  Length = space[1] - space[0]
  M = int(Length / h)
  n = M - 1

  # Construct B and A
  B = (np.diag(4.0 * np.ones(n), 0)  + np.diag(-1.0 * np.ones(n-1), 1) + np.diag(-1.0 * np.ones(n-1), -1))
  I = np.eye(n)
  C = np.diag(-1.0 * np.ones(n-1), 1) + np.diag(-1.0 * np.ones(n-1), -1)
  A = (np.kron(B, I) + np.kron(C, I))
  #print(A)
  f_vec = np.zeros(n*n)
  for i in range(n):
    for j in range(n):
      x1, x2 = (i + 1) * h, (j + 1) * h # get x1, x2 for h, 2h,...., (M-1)h
      f_vec[j * n + i] = load_vec_lattices(i, j, h, n) * h**2 / 2
  estimated_u = np.linalg.solve(A, f_vec)
  estimated_u = estimated_u.reshape((n, n))
  return estimated_u


In [203]:
def vectorize_u(h, space):
  Length = space[1] - space[0]
  M = int(Length / h)
  n = M - 1
  u_vec = np.zeros(n*n)
  for i in range(n):
    for j in range(n):
      x1, x2 = (i + 1) * h, (j + 1) * h
      u_vec[j * n + i] = u(x1, x2)
  u_vec = u_vec.reshape((n, n))
  return u_vec

In [204]:
h1 = 1/10
h2 = 1/20
space = [0, 1]

# h = 1/10
Uj1 = Finite_Difference_Approx(h1, space)

# h = 1/20
Uj2 = Finite_Difference_Approx(h2, space)

In [205]:
u1-Uj1

array([[ 0.0087042 ,  0.01031319,  0.01091265,  0.0104439 ,  0.00895283,
         0.0065854 ,  0.00357334,  0.0002115 , -0.00317105],
       [ 0.01536804,  0.01689842,  0.01677465,  0.01500887,  0.01177392,
         0.00738645,  0.00227594, -0.00305735, -0.00809136],
       [ 0.01871846,  0.0186385 ,  0.01673408,  0.0131916 ,  0.00835784,
         0.00270596, -0.0032108 , -0.00881327, -0.01355303],
       [ 0.01847748,  0.01589898,  0.01176418,  0.00647782,  0.00055737,
        -0.00541765, -0.01086235, -0.01524376, -0.01813301],
       [ 0.01537427,  0.01032707,  0.00426899, -0.00220697, -0.0084669 ,
        -0.01389802, -0.01796872, -0.02028051, -0.0206071 ],
       [ 0.01080819,  0.0042031 , -0.00281342, -0.00955454, -0.0153604 ,
        -0.01966267, -0.02204023, -0.02226033, -0.02030144],
       [ 0.00629728, -0.00040597, -0.00706948, -0.01304098, -0.01773593,
        -0.02069477, -0.02162787, -0.02044387, -0.01725869],
       [ 0.00292449, -0.00236994, -0.00743239, -0.0117673 , -0

In [206]:
u1 = vectorize_u(h1, space)
u2 = vectorize_u(h2, space)

# L2 Norm of the error for $h=1/10$: 0.03593

In [207]:
# calculate L-2 Norm
def norm(x, h):
  return np.sqrt(np.sum(x**2 * h**2/2))
l2_norm1 = norm(Uj1 - u1, h1)
l2_norm1

0.00800926889189662

# L2 Norm of the error for $h=1/20$: 0.07495

In [208]:
l2_norm2 = norm(Uj2 - u2, h2)
l2_norm2

0.005017275971481492